# Preprocessing (for U-Net)
This notebook contains imutils to prepare data to Train Unet.
#Or other things too 

In [ ]:
import tifffile as tiff
from natsort import natsorted
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
from imutils.src import imfunctions #contours_length as imutils.contours_length
from natsort import natsorted
import shutil

#### Part 1: Extract frames


In [ ]:
#Here I am extracting frames from a specific interval

frames2extract=np.arange(0,200000,17777)#(0,200000,10000)
input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all/'
output_folder='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
try: os.mkdir(output_folder) # creates the subdirectory where it should be stored
except: print('Output Directory already exists, might overwrite')

for btf_file in natsorted(os.listdir(input_folder)):
    print(btf_file)
    if '2020-07' in btf_file:
        print(btf_file)
        btf_file_basename=os.path.splitext(os.path.basename(btf_file))[0]
        print(btf_file_basename)
        imfunctions.extract_frames(os.path.join(input_folder,btf_file), os.path.join(output_folder,btf_file_basename),frames2extract)
        print('end')

In [ ]:
#add zeros (just in case)
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
for dirpath, _, files in os.walk(path):
    print(dirpath)
    imfunctions.add_zeros_to_filename(dirpath, 6)

In [ ]:
#rename to have unique names (prepend folder name into file name)
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
for dirpath, _, files in os.walk(path):
    for f in files:
        os.rename(os.path.join(dirpath, f), os.path.join(dirpath, os.path.split(dirpath)[-1] + '_' + f))

In [ ]:
#copy all files from subfolders into a new single folder
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
dst_path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test_all/'

try: os.mkdir(dst_path) # creates the subdirectory where it should be stored
except: print('Output Directory already exists, might overwrite')

    for file in files:
        #print(file)
        shutil.copyfile(os.path.join(directory,file),os.path.join(dst_path,file))


In [ ]:
#substract background if wanted

bg_img_filename='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/background/2020-07-01_13-42-05_chemotaxisl_worm1_bg-channel-0-_MMStack_Median_BG.tiff'
bg_img=tiff.imread(bg_img_filename)

path='/groups/zimmer/Ulises/code/unet-master/data/worm2/test2/'

#this code is flexible to files being in separate folders
for directory, subdirectories, files in os.walk(path):
    for file in files:
        filepath=os.path.join(directory,file)
        img=tiff.imread(filepath)
        inv_img=cv2.bitwise_not(img)
        new_img=cv2.subtract(inv_img,bg_img)
        tiff.imwrite(filepath,new_img)

In [ ]:
#Use draw_some_contours function to extract interesting contours
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_all_image/'

for directory, subdirectories, files in os.walk(path):
    for file in files:
        filepath=os.path.join(directory,file)
        img=tiff.imread(filepath)
        #median Blur
        img=cv2.medianBlur(img,5)
       
        #apply threshold
        ret, new_img = cv2.threshold(img,20,255,cv2.THRESH_BINARY)
        worm_contour_img=imfunctions.draw_some_contours(new_img,contour_size=8000,tolerance=0.25, inner_contour_area_to_fill=300)
        
        #if you want to plot the different stages
        #plotting
#         fig, (ax1,ax2)=plt.subplots(figsize=(10,5), nrows=1, ncols=2, dpi=100)
#         im=ax1.imshow(img)
#         divider = make_axes_locatable(ax1)
#         cax = divider.append_axes('right', size='5%', pad=0.05)
#         fig.colorbar(im, cax=cax, orientation='vertical')
#         ax2.imshow(worm_contour_img)
#         [axi.set_axis_off() for axi in (ax1,ax2)]
        
        tiff.imwrite(filepath,worm_contour_img)

In [ ]:
#convert to stack if wanted to screen for masks that need improvement (eg remove bubble)
path='/groups/zimmer/Ulises/code/unet-master/data/new_training_all_image/'
o_path='/groups/zimmer/Ulises/code/unet-master/data/new_training_all_stack.tif'
imfunctions.images2stack(path, o_path)

In [ ]:
#manually annotate frames to correct
path='/groups/zimmer/Ulises/code/unet-master/data/worm2_background_removed/test/'
idxs=[53,76,98]#[15,67,80,89,92,113,157,158,169]
#get their filename
for idx,file in enumerate(natsorted(os.listdir(path))):
    #add +1 if idxs are starting from 1
    if idx+1 in idxs:
        print(idx+1, file)

#### Part 2
Extract only frames where there is a coiled shape (contour have children)

**Note**: There is also the function find_nan_centerlines() in the centerline package, which outputs the selected frames that then can be extracted with extract_frames()

In [ ]:
#for very file in binary_all_folder:
main_input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary/'
output_folder='/groups/zimmer/Ulises/code/unet-master/data/crop_worm_raw/train/label/'

for input_filepath in os.listdir(main_input_folder):
    #if input_filepath.endswith('.*tif*'):# and '2020' in input_filepath:
    if '2020-07' in input_filepath:
        file_basename=os.path.splitext(os.path.basename(input_filepath))[0]
        with tiff.TiffFile(os.path.join(main_input_folder,input_filepath)) as tif:
            for i, page in enumerate(tif.pages):#enumerate(tif.series):#enumerate(tif.series[0].pages):
                if i%1000!=0: continue
                img=page.asarray()
        #         plt.imshow(img)
        #         plt.show()
                contours_with_children=imfunctions.extract_contours_with_children(img)

                for cnt_idx,cnt in enumerate(contours_with_children):
                        x,y,w,h = cv2.boundingRect(cnt)
                        #make the crop
                        cnt_img=img[y:y+h,x:x+w]
        #                 plt.imshow(cnt_img)
        #                 plt.show()
                        tiff.imwrite(os.path.join(output_folder,file_basename+'_frame_'+str(i)+'_cnt_'+str(cnt_idx)+'.tiff'),cnt_img)
                #if i>20000: break
        

In [ ]:
#for very file in btf_all_folder: Using raw data
#TO DO: Make it more modular
main_input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_background_substracted//'
output_folder='/groups/zimmer/Ulises/code/unet-master/data/crop_worm_raw/'

for input_filepath in os.listdir(main_input_folder):
    #if input_filepath.endswith('.*tif*'):# and '2020' in input_filepath:
    if '2020-07' in input_filepath:
        file_basename=os.path.splitext(os.path.basename(input_filepath))[0]
        with tiff.TiffFile(os.path.join(main_input_folder,input_filepath)) as tif:
            for i, page in enumerate(tif.pages):#enumerate(tif.series):#enumerate(tif.series[0].pages):
                if i%1000!=0: continue
                img=page.asarray()

                #median Blur
                img_blurred=cv2.medianBlur(img,5)
               
                #apply threshold
                ret, new_img = cv2.threshold(img_blurred,20,255,cv2.THRESH_BINARY)
                #draw_some_contours does not need imfunctions.draw_some_contours in here. But outside this file.
                worm_contour_img=imfunctions.draw_some_contours(new_img,contour_size=8000,tolerance=0.25, inner_contour_area_to_fill=300)
            
                
                contours_with_children=imfunctions.extract_contours_with_children(worm_contour_img)

                for cnt_idx,cnt in enumerate(contours_with_children):
                        x,y,w,h = cv2.boundingRect(cnt)
                        #make the crop
                        cnt_img=img[y:y+h,x:x+w]
        #                 plt.imshow(cnt_img)
        #                 plt.show()
                        tiff.imwrite(os.path.join(output_folder,file_basename+'_frame_'+str(i)+'_cnt_'+str(cnt_idx)+'.tiff'),cnt_img)
                #if i>20000: break
        

In [ ]:
img_path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm_raw/train/label/2020-07-01_17-32-12_control_worm4-channel-0-bigtiff_frame_60000_raw_cnt_0.tiff'
img=tiff.imread(img_path)
plt.imshow(img)

In [ ]:
## Compare image vs label
train_folder='/groups/zimmer/Ulises/code/unet-master/data/crop_worm/train/'
image_path=os.path.join(train_folder,'image')
label_path=os.path.join(train_folder,'label')

for file in natsorted(os.listdir(image_path)):
    #print(file)
    fig, (ax1,ax2)=plt.subplots(ncols=2)
    image=tiff.imread(os.path.join(image_path, file))
    label=tiff.imread(os.path.join(label_path, file))

    ax1.imshow(image)
    ax2.imshow(label)
    fig.suptitle(file)
    

### Part 3
Cut everywhere where there are contours with children, run the network and paste them back

In [ ]:
import tifffile as tiff
from natsort import natsorted
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
from imutils.src import imfunctions #contours_length as imutils.contours_length
from natsort import natsorted
import shutil

from imutils.src.model import *

In [ ]:
#for very file in binary_all_folder:
main_input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary/'
output_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary_after_unet/'

weights_path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm/training_results/24051180_w_validation500steps_10epochs/unet_master.hdf5'

#unet has to have been imported
model=unet()
model.load_weights(weights_path)

for input_filepath in os.listdir(main_input_folder):
    #if input_filepath.endswith('.*tif*'):# and '2020' in input_filepath:
    if '2020-07' in input_filepath:
        file_basename=os.path.splitext(os.path.basename(input_filepath))[0]
        with tiff.TiffFile(os.path.join(main_input_folder,input_filepath)) as tif:
            for i, page in enumerate(tif.pages):#enumerate(tif.series):#enumerate(tif.series[0].pages):
                if i>30000:break
                if i%50!=0: continue
                img=page.asarray()
        #         plt.imshow(img)
        #         plt.show()
                contours_with_children=imfunctions.extract_contours_with_children(img)

                for cnt_idx,cnt in enumerate(contours_with_children):
                        x,y,w,h = cv2.boundingRect(cnt)
                        #make the crop
                        cnt_img=img[y:y+h,x:x+w]
                        
                        #run U-Net network:
                        if cnt_img.shape!=(256,256):
                            cnt_img=cv2.resize(cnt_img, (256,256))
                            print('original image has been resized to: ', cnt_img.shape)
                        cnt_img=np.reshape(cnt_img,cnt_img.shape+(1,))
                        cnt_img=np.reshape(cnt_img,(1,)+cnt_img.shape)

                        print('New image shape is: ', cnt_img.shape)
                        cnt_img=cnt_img/255
                        results = model.predict(cnt_img)
                        print('Results image shape is: ', results.shape)
                        #reshape results
                        results_reshaped=results.reshape(256,256)
                        #resize results
                        results_reshaped=cv2.resize(results_reshaped, (w,h))
                        #multiply it by 255
                        results_reshaped=results_reshaped*255
                        
                        #paste it into the original image
                        new_img=img.copy()
                        new_img[y:y+h,x:x+w]=results_reshaped
                        
                        #visualize
#                         fig,(ax0,ax1,ax2,ax3,ax4,ax5)=plt.subplots(figsize=(18,3), ncols=5)
#                         ax0.imshow(img)
#                         ax1.imshow(img[y:y+h,x:x+w])
#                         ax2.imshow(cnt_img.reshape((256,256)))
#                         ax3.imshow(results.reshape((256,256)))
#                         ax4.imshow(results_reshaped)
#                         ax5.imshow(img)
#                         plt.show()
#                         fig,(ax0,ax1)=plt.subplots(figsize=(6,3), ncols=2,dpi=150)
#                         ax0.imshow(img)
#                         ax1.imshow(new_img, vmin=240)
#                         plt.show()
                        
#                         break
        #                 plt.imshow(cnt_img)
        #                 plt.show()
                        tiff.imwrite(os.path.join(output_folder,file_basename+'_frame_'+str(i)+'_cnt_'+str(cnt_idx)+'.tiff'),cnt_img)
                #if i>20000: break

In [ ]:
#with a tiff writer (it exists as a function in imfunctions)
#for very file in binary_all_folder:
main_input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary_after_unet/'
output_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary_after_unet2/'

weights_path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm/training_results/24051180_w_validation500steps_10epochs/unet_master.hdf5'

#unet has to have been imported
model=unet()
model.load_weights(weights_path)

for input_filepath in os.listdir(main_input_folder):
    #if input_filepath.endswith('.*tif*'):# and '2020' in input_filepath:
    if '2020-07' in input_filepath:
        file_basename=os.path.splitext(os.path.basename(input_filepath))[0]

        #starts tiff writer and tiff reader
        with tiff.TiffWriter(os.path.join(output_folder, input_filepath), bigtiff=True) as tif_writer:
            with tiff.TiffFile(os.path.join(main_input_folder,input_filepath)) as tif:
                for i, page in enumerate(tif.pages):
                    img=page.asarray()
                    
                    #find contours with children
                    contours_with_children=imfunctions.extract_contours_with_children(img)

                    #make a copy of the original image here in order to paste more than one contour with children
                    new_img=img.copy()
                    for cnt_idx,cnt in enumerate(contours_with_children):
                            x,y,w,h = cv2.boundingRect(cnt)
                            #make the crop
                            cnt_img=img[y:y+h,x:x+w]

                            #run U-Net network:
                            cnt_img=cv2.resize(cnt_img, (256,256))
                            cnt_img=np.reshape(cnt_img,cnt_img.shape+(1,))
                            cnt_img=np.reshape(cnt_img,(1,)+cnt_img.shape)

                            #normalize to 1 by dividing by 255
                            cnt_img=cnt_img/255
                            results = model.predict(cnt_img)
                            #reshape results
                            results_reshaped=results.reshape(256,256)
                            #resize results
                            results_reshaped=cv2.resize(results_reshaped, (w,h))
                            #multiply it by 255
                            results_reshaped=results_reshaped*255

                            #paste it into the original image
                            new_img[y:y+h,x:x+w]=results_reshaped

                    tif_writer.write(new_img, contiguous=True)

### Trying eroding mask

In [ ]:
from skimage.morphology import binary_erosion
import tifffile as tiff


In [ ]:
np.ones(3,3)

In [ ]:

path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm_eroded/train/label/'
eroded_path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm_eroded/train/label_eroded2/'

for file in (os.listdir(path)):
    img=tiff.imread(os.path.join(path,file))
    eroded=binary_erosion(img)
    eroded=binary_erosion(eroded)
    #conver to image with values form 0 to 255
    eroded = eroded.astype(np.uint8)  #convert to an unsigned byte
    eroded*=255
    fig, (ax0,ax1)=plt.subplots(ncols=2)
    ax0.imshow(img)
    ax1.imshow(eroded)
    tiff.imwrite(os.path.join(eroded_path,file), eroded)
    

In [ ]:
os.listdir(path)